**대규모 텐서플로 모델 훈련과 배포**

# 설정

In [1]:
from tensorflow import config

config.list_physical_devices('GPU')

2025-08-18 02:44:27.470533: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755485067.484535    1918 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755485067.488908    1918 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755485067.502110    1918 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755485067.502124    1918 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755485067.502126    1918 computation_placer.cc:177] computation placer alr

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# 텐서플로 모델 서빙하기

먼저 TF 서빙을 사용하여 모델을 배포한 다음 Google Vertex AI에 배포해 본다.

## 텐서플로 서빙 사용

가장 먼저 모델을 빌드하고 학습한 다음 SavedModel 포맷으로 내보낸다.

### SavedModel 내보내기

MNIST 데이터 세트를 로드하고, 스케일을 조정하고, 분할한다.

In [2]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras import backend, layers
from tensorflow import keras
import tensorflow as tf
from pathlib import Path

mnist_ = mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = mnist_
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
backend.clear_session()
model = keras.Sequential()
model.add(keras.Input((28, 28), dtype=tf.uint8))
model.add(layers.Flatten())
model.add(layers.Rescaling(1 / 255))
model.add(layers.Dense(100, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
model.compile(optimizer='SGD', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))
model_name = 'my_mnist_model'
model_version = '0001'
model_path = Path(model_name) / model_version
model.export(model_path)

I0000 00:00:1755485069.486324    1918 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5557 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3070 Ti, pci bus id: 0000:06:00.0, compute capability: 8.6


Epoch 1/10


I0000 00:00:1755485070.662537    1968 service.cc:152] XLA service 0x7bae14005bf0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1755485070.662577    1968 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 3070 Ti, Compute Capability 8.6
2025-08-18 02:44:30.674189: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1755485070.709426    1968 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-08-18 02:44:31.450709: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_36_0', 92 bytes spill stores, 96 bytes spill loads

2025-08-18 02:44:31.883314: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_36

  77/1719 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.1856 - loss: 2.2232

I0000 00:00:1755485073.888349    1968 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


...

1719/1719 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7101 - loss: 1.0946

2025-08-18 02:44:41.837208: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_34', 8 bytes spill stores, 8 bytes spill loads

2025-08-18 02:44:41.890388: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_34', 296 bytes spill stores, 296 bytes spill loads

2025-08-18 02:44:41.939128: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_34', 8 bytes spill stores, 8 bytes spill loads

2025-08-18 02:44:42.024890: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_34', 792 bytes spill stores, 844 bytes spill loads

2025-08-18 02:44:42.250937: I external/local_xla

1719/1719 ━━━━━━━━━━━━━━━━━━━━ 12s 5ms/step - accuracy: 0.7102 - loss: 1.0943 - val_accuracy: 0.9000 - val_loss: 0.3708
Epoch 2/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8950 - loss: 0.3693 - val_accuracy: 0.9188 - val_loss: 0.3006
Epoch 3/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9140 - loss: 0.3051 - val_accuracy: 0.9282 - val_loss: 0.2673
Epoch 4/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9197 - loss: 0.2853 - val_accuracy: 0.9348 - val_loss: 0.2455
Epoch 5/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9314 - loss: 0.2486 - val_accuracy: 0.9370 - val_loss: 0.2252
Epoch 6/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9343 - loss: 0.2331 - val_accuracy: 0.9424 - val_loss: 0.2116
Epoch 7/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9410 - loss: 0.2139 - val_accuracy: 0.9458 - val_loss: 0.1997
Epoch 8/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9430 - loss: 0.2036 - val_accura

INFO:tensorflow:Assets written to: my_mnist_model/0001/assets


Saved artifact at 'my_mnist_model/0001'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28), dtype=tf.uint8, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  135991302106832: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135991302109136: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135991302107984: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135991302109712: TensorSpec(shape=(), dtype=tf.resource, name=None)


파일 트리를 살핀다:

In [4]:
sorted([str(path) for path in model_path.parent.glob('**/*')])

['my_mnist_model/0001',
 'my_mnist_model/0001/assets',
 'my_mnist_model/0001/fingerprint.pb',
 'my_mnist_model/0001/saved_model.pb',
 'my_mnist_model/0001/variables',
 'my_mnist_model/0001/variables/variables.data-00000-of-00001',
 'my_mnist_model/0001/variables/variables.index']

SavedModel을 검사한다:

In [5]:
!saved_model_cli show --dir '{model_path}'

2025-08-18 02:45:19.010531: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755485119.025760    2784 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755485119.034577    2784 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755485119.053987    2784 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755485119.054027    2784 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755485119.054034    2784 computation_placer.cc:177] computation placer alr

In [6]:
!saved_model_cli show --dir '{model_path}' --tag_set serve

2025-08-18 02:45:21.699076: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755485121.712145    2800 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755485121.716310    2800 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755485121.727815    2800 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755485121.727851    2800 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755485121.727857    2800 computation_placer.cc:177] computation placer alr

In [7]:
!saved_model_cli show --dir '{model_path}' --tag_set serve --signature_def serving_default

2025-08-18 02:45:24.400845: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755485124.414182    2817 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755485124.418509    2817 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755485124.430006    2817 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755485124.430041    2817 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755485124.430047    2817 computation_placer.cc:177] computation placer alr

더 자세한 내용을 보려면 다음 명령을 실행한다:
```ipython
!saved_model_cli show --dir '{model_path}' --all
```

### 텐서플로 서빙 설치 및 시작하기

도커를 사용해 TF 서빙을 설치하려면 먼저 [Docker](https://docs.docker.com/install/)가 설치되어 있는지 확인한 후 터미널에서 다음 명령을 실행한다.
```bash
docker pull tensorflow/serving:latest-gpu

docker run -e MODEL_NAME=my_mnist_model --gpus all -it -p 8500:8500 -p 8501:8501 -v path\to\my_mnist_model:/models/my_mnist_model tensorflow/serving:latest-gpu
```

### REST API로 TF 서빙에 쿼리하기

다음으로 TF 서빙에 REST 쿼리를 전송한다:

In [8]:
import json

X_new = X_test[:3]  # 분류할 새로운 숫자 이미지가 3개 있다고 가정한다.
request_json = json.dumps({'signature_name': 'serving_default', 'instances': X_new.tolist()})

In [9]:
request_json[:100] + '...' + request_json[-10:]

'{"signature_name": "serving_default", "instances": [[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..., 0, 0]]]}'

이제 텐서플로 서빙의 REST API를 사용하여 예측한다:

In [10]:
import requests

server_url = 'http://172.17.0.3:8501/v1/models/my_mnist_model:predict'
response = requests.post(server_url, request_json)
response.raise_for_status()  # 오류 발생 시 예외 발생
response = response.json()

In [11]:
import numpy as np

y_proba = np.array(response['predictions'])
y_proba.round(2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.99, 0.  , 0.  ],
       [0.  , 0.  , 0.97, 0.02, 0.  , 0.  , 0.01, 0.  , 0.  , 0.  ],
       [0.  , 0.98, 0.01, 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.  ]])

### gRPC API로 TF 서빙에 쿼리하기

In [12]:
from tensorflow_serving.apis.predict_pb2 import PredictRequest

request = PredictRequest()
request.model_spec.name = model_name
request.model_spec.signature_name = 'serving_default'
input_name = 'keras_tensor'
request.inputs[input_name].CopyFrom(tf.make_tensor_proto(X_new))

In [13]:
request

model_spec {
  name: "my_mnist_model"
  signature_name: "serving_default"
}
inputs {
  key: "keras_tensor"
  value {
    dtype: DT_UINT8
    tensor_shape {
      dim {
        size: 3
      }
      dim {
        size: 28
      }
      dim {
        size: 28
      }
    }
    tensor_content: "\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\00

In [14]:
import grpc
from tensorflow_serving.apis import prediction_service_pb2_grpc

channel = grpc.insecure_channel('172.17.0.3:8500')
predict_service = prediction_service_pb2_grpc.PredictionServiceStub(channel)
response = predict_service.Predict(request, timeout=10.0)

응답을 텐서로 변환한다:

In [15]:
output_name = 'output_0'
outputs_proto = response.outputs[output_name]
y_proba = tf.make_ndarray(outputs_proto)

In [16]:
y_proba.round(2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.99, 0.  , 0.  ],
       [0.  , 0.  , 0.97, 0.02, 0.  , 0.  , 0.01, 0.  , 0.  , 0.  ],
       [0.  , 0.98, 0.01, 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.  ]],
      dtype=float32)

### 새 모델 버전 배포

In [17]:
# 새로운 MNIST 모델 버전 빌드 및 훈련
model = keras.Sequential()
model.add(keras.Input((28, 28), dtype=tf.uint8))
model.add(layers.Flatten())
model.add(layers.Rescaling(1 / 255))
model.add(layers.Dense(50, activation='relu'))
model.add(layers.Dense(50, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
model.compile(optimizer='SGD', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

Epoch 1/10


2025-08-18 02:45:29.992254: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_38', 272 bytes spill stores, 272 bytes spill loads

2025-08-18 02:45:30.428816: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_38', 944 bytes spill stores, 944 bytes spill loads

2025-08-18 02:45:30.580719: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_38', 940 bytes spill stores, 940 bytes spill loads

2025-08-18 02:45:30.657226: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_38', 1180 bytes spill stores, 1240 bytes spill loads

2025-08-18 02:45:31.300613: I external

1713/1719 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6068 - loss: 1.2931

2025-08-18 02:45:36.870446: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_38', 272 bytes spill stores, 272 bytes spill loads

2025-08-18 02:45:37.203220: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_38', 944 bytes spill stores, 944 bytes spill loads

2025-08-18 02:45:37.240944: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_38', 988 bytes spill stores, 988 bytes spill loads

2025-08-18 02:45:37.389117: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_38', 956 bytes spill stores, 956 bytes spill loads

2025-08-18 02:45:37.445854: I external/l

1719/1719 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6074 - loss: 1.2913

2025-08-18 02:45:40.760795: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_36', 272 bytes spill stores, 272 bytes spill loads

2025-08-18 02:45:40.787464: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_36', 348 bytes spill stores, 348 bytes spill loads

2025-08-18 02:45:40.981013: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_36', 816 bytes spill stores, 816 bytes spill loads

2025-08-18 02:45:41.049617: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_36', 272 bytes spill stores, 272 bytes spill loads



1719/1719 ━━━━━━━━━━━━━━━━━━━━ 12s 5ms/step - accuracy: 0.6075 - loss: 1.2910 - val_accuracy: 0.8984 - val_loss: 0.3651
Epoch 2/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8977 - loss: 0.3541 - val_accuracy: 0.9212 - val_loss: 0.2833
Epoch 3/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9184 - loss: 0.2856 - val_accuracy: 0.9344 - val_loss: 0.2424
Epoch 4/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9274 - loss: 0.2532 - val_accuracy: 0.9402 - val_loss: 0.2178
Epoch 5/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9355 - loss: 0.2240 - val_accuracy: 0.9438 - val_loss: 0.1948
Epoch 6/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9436 - loss: 0.1988 - val_accuracy: 0.9496 - val_loss: 0.1767
Epoch 7/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9478 - loss: 0.1821 - val_accuracy: 0.9516 - val_loss: 0.1662
Epoch 8/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9536 - loss: 0.1683 - val_accura

In [18]:
model_version = '0002'
model_path = Path(model_name) / model_version
model.export(model_path)

INFO:tensorflow:Assets written to: my_mnist_model/0002/assets


INFO:tensorflow:Assets written to: my_mnist_model/0002/assets


Saved artifact at 'my_mnist_model/0002'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28), dtype=tf.uint8, name='keras_tensor_11')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  135991298675472: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135991298679120: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135991298682384: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135991298685264: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135991298684112: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135991298674512: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [19]:
sorted([str(path) for path in model_path.parent.glob('**/*')])

['my_mnist_model/0001',
 'my_mnist_model/0001/assets',
 'my_mnist_model/0001/fingerprint.pb',
 'my_mnist_model/0001/saved_model.pb',
 'my_mnist_model/0001/variables',
 'my_mnist_model/0001/variables/variables.data-00000-of-00001',
 'my_mnist_model/0001/variables/variables.index',
 'my_mnist_model/0002',
 'my_mnist_model/0002/assets',
 'my_mnist_model/0002/fingerprint.pb',
 'my_mnist_model/0002/saved_model.pb',
 'my_mnist_model/0002/variables',
 'my_mnist_model/0002/variables/variables.data-00000-of-00001',
 'my_mnist_model/0002/variables/variables.index']

**경고**: 텐서플로 서빙이 새 모델을 로드하기까지 잠시 기다려야 할 수 있다.

In [20]:
server_url = 'http://172.17.0.3:8501/v1/models/my_mnist_model:predict'
response = requests.post(server_url, request_json)
response.raise_for_status()
response = response.json()

In [21]:
response.keys()

dict_keys(['predictions'])

In [22]:
y_proba = np.array(response['predictions'])
y_proba.round(2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.99, 0.  , 0.  ],
       [0.  , 0.  , 0.97, 0.02, 0.  , 0.  , 0.01, 0.  , 0.  , 0.  ],
       [0.  , 0.98, 0.01, 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.  ]])

# 모바일 또는 임베디드 디바이스에 모델 배포하기

In [23]:
from tensorflow import lite

converter = lite.TFLiteConverter.from_saved_model(str(model_path))
tflite_model = converter.convert()
with open('my_converted_savedmodel.tflite', 'wb') as f:
    f.write(tflite_model)

W0000 00:00:1755485182.547978    1918 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1755485182.548013    1918 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-08-18 02:46:22.551550: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: my_mnist_model/0002
2025-08-18 02:46:22.554622: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-08-18 02:46:22.554634: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: my_mnist_model/0002
I0000 00:00:1755485182.558779    1918 mlir_graph_optimization_pass.cc:425] MLIR V1 optimization pass is not enabled
2025-08-18 02:46:22.559443: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-08-18 02:46:22.584923: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: my_mnist_model/0002
2025-08-18 02:46:22.589875: I tensorflow/cc/saved_model/loader.cc:471] Sa

In [24]:
# 케라스 모델을 변환한다.
converter = lite.TFLiteConverter.from_keras_model(model)

In [25]:
converter.optimizations = [lite.Optimize.DEFAULT]

In [26]:
tflite_model = converter.convert()
with open('my_converted_keras_model.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpx8p9dgp3/assets


INFO:tensorflow:Assets written to: /tmp/tmpx8p9dgp3/assets


Saved artifact at '/tmp/tmpx8p9dgp3'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28), dtype=tf.uint8, name='keras_tensor_11')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  135991298675472: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135991298679120: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135991298682384: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135991298685264: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135991298684112: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135991298674512: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1755485182.882199    1918 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1755485182.882241    1918 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-08-18 02:46:22.882368: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpx8p9dgp3
2025-08-18 02:46:22.882745: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-08-18 02:46:22.882751: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmpx8p9dgp3
2025-08-18 02:46:22.885286: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-08-18 02:46:22.899809: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmpx8p9dgp3
2025-08-18 02:46:22.904604: I tensorflow/cc/saved_model/loader.cc:471] SavedModel load for tags { serve }; Status: success: OK. Took 22239 microseconds.


# GPU를 사용하여 계산 속도 향상하기

텐서플로우가 GPU를 볼 수 있는지 확인한다:

In [27]:
physical_gpus = config.list_physical_devices('GPU')
physical_gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

텐서플로 스크립트에서 GPU \#0과 \#1(PCI 순서 기준)만 사용하려면 스크립트를 시작하기 전에 환경 변수 `CUDA_DEVICE_ORDER=PCI_BUS_ID`와 `CUDA_VISIBLE_DEVICES=0,1`을 설정한다. 또는 스크립트 자체에서 텐서플로를 사용하기 전에 지정한다.

## GPU RAM 관리

RAM 용량을 GPU당 2GB로 제한하려면:

In [28]:
# for gpu in physical_gpus:
#     config.set_logical_device_configuration(gpu, [config.LogicalDeviceConfiguration(2048)])

점진적으로 텐서플로가 메모리를 점유하도록 하려면(프로세스가 종료될 때만 메모리를 해제한다):

In [29]:
# from tensorflow.config import experimental

# for gpu in physical_gpus:
#     experimental.set_memory_growth(gpu, True)

이와 동일하게, 텐서플로를 사용하기 전에 `TF_FORCE_GPU_ALLOW_GROWTH` 환경 변수를 `true`로 설정할 수 있다.

물리적 GPU를 두 개의 논리적 GPU로 분할한다:

In [30]:
# config.set_logical_device_configuration(
#     physical_gpus[0], [config.LogicalDeviceConfiguration(2048), config.LogicalDeviceConfiguration(2048)]
# )

In [31]:
logical_gpus = config.list_logical_devices('GPU')
logical_gpus

[LogicalDevice(name='/device:GPU:0', device_type='GPU')]

## 디바이스에 연산 및 변수 배치하기

모든 변수 및 연산 배치를 기록하려면(이 작업은 텐서플로를 임포팅한 직후에 실행해야 한다):

In [32]:
from tensorflow import debugging

tf.get_logger().setLevel("DEBUG")  # 디폴트 로그 수준은 INFO다.
debugging.set_log_device_placement(True)

In [33]:
a = tf.Variable([1., 2., 3.])  # float32 변수는 GPU로 이동한다.
a.device

'/job:localhost/replica:0/task:0/device:GPU:0'

In [34]:
b = tf.Variable([1, 2, 3])  # int32 변수는 CPU로 이동한다.
b.device

'/job:localhost/replica:0/task:0/device:CPU:0'

`tf.device()` 컨텍스트를 사용하여 원하는 장치에 변수 및 연산을 수동으로 배치할 수 있다:

In [35]:
with tf.device('/cpu:0'):
    c = tf.Variable([1., 2., 3.])
c.device

'/job:localhost/replica:0/task:0/device:CPU:0'

존재하지 않거나 커널이 없는 장치를 지정하면 텐서플로는 자동으로 기본 장치를 사용한다:

In [36]:
with tf.device('/gpu:1234'):
    d = tf.Variable([1., 2., 3.])
d.device

'/job:localhost/replica:0/task:0/device:GPU:0'

텐서플로에서 존재하지 않는 장치를 사용하려고 할 때 기본 장치로 되돌아가지 않고 예외를 발생시키려면:

In [37]:
from tensorflow import errors

config.set_soft_device_placement(False)
try:
    with tf.device('/gpu:1000'):
        d = tf.Variable([1., 2., 3.])
except errors.InvalidArgumentError as ex:
    print(ex)
config.set_soft_device_placement(True)  # 소프트 배치로 돌아가기

Could not satisfy device specification '/job:localhost/replica:0/task:0/device:GPU:1000'. enable_soft_placement=0. Supported device types [GPU, CPU]. All available devices [/job:localhost/replica:0/task:0/device:GPU:0, /job:localhost/replica:0/task:0/device:CPU:0].


## 여러 디바이스에서 병렬 실행

inter-op 또는 intra-op 스레드 수를 설정하려는 경우(CPU 포화를 방지하거나 완벽하게 재현 가능한 테스트 케이스를 실행하기 위해 텐서플로를 단일 스레드로 만들고자 하는 경우 유용하다):

In [41]:
from tensorflow.config import threading

# threading.set_inter_op_parallelism_threads(10)
# threading.set_intra_op_parallelism_threads(10)